In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
minor_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)
minor_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240318상장비주력test_stand.csv', index_col=0)

##### train_df, test_df 설정

In [ ]:
train_df = minor_s[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [ ]:
test_df = minor_test[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [ ]:
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [ ]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성
under = RandomUnderSampler(random_state=4, sampling_strategy=0.1)

# RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
x_resampled, y_resampled = under.fit_resample(x_train, y_train)

In [ ]:
# x_resampled, y_resampled = RandomOverSampler(random_state=4).fit_resample(x_train,y_train)
# print(x_train.shape)
# y_train.value_counts()

(3485, 13)


0.0    3431
1.0      54
Name: 부실, dtype: int64

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))


##### 1. RandomForest

In [ ]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=52)

rf_clf.fit(x_resampled, y_resampled)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [ ]:
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


##### 하이퍼파라미터 조정 & 임계값 설정한 뒤 모델링

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# RandomForestClassifier 모델 생성
rf_clf = RandomForestClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝과 교차 검증 수행
grid_search = GridSearchCV(rf_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 가져오기
best_rf_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
rf_pred = best_rf_clf.predict(x_test)

In [ ]:
# 임계값 설정
threshold = 0.6
rf_pred = (rf_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1622  109]
 [  12    7]]
정확도: 0.9309, 정밀도 : 0.0603, 재현율:0.3684,F1 스코어:0.1037
ROC 스코어: 0.6527, PR score : 0.0291


##### 2. LogisticRegression

In [ ]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(x_resampled, y_resampled)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)

In [ ]:
print("로지스틱")
get_clf_eval(y_test, lr_pred)

로지스틱
오차행렬
[[1119   44]
 [   0    5]]
정확도: 0.9623, 정밀도 : 0.1020, 재현율:1.0000,F1 스코어:0.1852
ROC 스코어: 0.9811, PR score : 0.1020


##### 3. SVM

In [ ]:
# SVM 모형 구현
from sklearn import svm

svm_clf = svm.SVC(kernel='linear', probability=True)  # 선형 SVM 모델 생성
svm_clf.fit(x_resampled, y_resampled)  # 훈련 데이터를 이용하여 모델 훈련
# 테스트 데이터를 사용하여 예측
svm_pred = svm_clf.predict(x_test)
svm_pred_proba = svm_clf.predict_proba(x_test)

In [ ]:
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1129   34]
 [   1    4]]
정확도: 0.9700, 정밀도 : 0.1053, 재현율:0.8000,F1 스코어:0.1860
ROC 스코어: 0.8854, PR score : 0.0851


##### 4. KNN

In [ ]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(x_resampled, y_resampled)
knn_pred = knn_clf.predict(x_test)
knn_pred_proba = knn_clf.predict_proba(x_test)

In [ ]:
print("knn")
get_clf_eval(y_test, knn_pred)

knn
오차행렬
[[1128   35]
 [   3    2]]
정확도: 0.9675, 정밀도 : 0.0541, 재현율:0.4000,F1 스코어:0.0952
ROC 스코어: 0.6850, PR score : 0.0242


##### 5. LGBM

In [ ]:
# LGBM 모형 구현
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()
lgb_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
lgb_pred = lgb_model.predict(x_test)
lgb_pred_proba = lgb_model.predict_proba(x_test)

[LightGBM] [Info] Number of positive: 3431, number of negative: 3431
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3066
[LightGBM] [Info] Number of data points in the train set: 6862, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [ ]:
print("LGBM")
get_clf_eval(y_test, lgb_pred)

LGBM
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


##### 6. xgboost

In [ ]:
# xgboost 모형 구현
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(x_resampled, y_resampled, verbose = True)


xgb_pred = xgb_wrapper.predict(x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(x_test)

In [ ]:
print("xgb")
get_clf_eval(y_test, xgb_pred)

xgb
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


#### 7. catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier()
catboost_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
catboost_pred = catboost_model.predict(x_test)
catboost_pred_proba = catboost_model.predict_proba(x_test)

Learning rate set to 0.023448
0:	learn: 0.6453004	total: 5.66ms	remaining: 5.65s
1:	learn: 0.6051456	total: 10.3ms	remaining: 5.13s
2:	learn: 0.5696390	total: 17.8ms	remaining: 5.92s
3:	learn: 0.5323364	total: 22.8ms	remaining: 5.68s
4:	learn: 0.5003691	total: 27.9ms	remaining: 5.56s
5:	learn: 0.4720252	total: 33ms	remaining: 5.46s
6:	learn: 0.4458052	total: 37.9ms	remaining: 5.38s
7:	learn: 0.4183371	total: 45.5ms	remaining: 5.64s
8:	learn: 0.3970178	total: 50.7ms	remaining: 5.58s
9:	learn: 0.3786603	total: 55.9ms	remaining: 5.53s
10:	learn: 0.3583645	total: 61.2ms	remaining: 5.5s
11:	learn: 0.3398555	total: 66.3ms	remaining: 5.46s
12:	learn: 0.3263700	total: 71.6ms	remaining: 5.43s
13:	learn: 0.3116138	total: 76.8ms	remaining: 5.41s
14:	learn: 0.2976417	total: 82ms	remaining: 5.38s
15:	learn: 0.2852332	total: 89.4ms	remaining: 5.5s
16:	learn: 0.2713522	total: 94.5ms	remaining: 5.46s
17:	learn: 0.2606646	total: 99.5ms	remaining: 5.43s
18:	learn: 0.2497275	total: 105ms	remaining: 5.41s

In [ ]:
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043
